In [0]:
inpatient = spark.read.parquet("/mnt/bronze/inpatient/inpatient_claims.parquet")

In [0]:
from pyspark.sql.functions import col, to_date, year, when, lit

def standardize_columns(df):
    return df.toDF(*[c.lower() for c in df.columns])
# Clean Inpatient

inpatient_clean = (
    inpatient.dropDuplicates(["DESYNPUF_ID", "CLM_ID"])
    .withColumn("CLM_ADMSN_DT", to_date(col("CLM_ADMSN_DT"), "yyyyMMdd"))
    .withColumn("CLM_FROM_DT", to_date(col("CLM_FROM_DT"), "yyyyMMdd"))
    .withColumn("CLM_THRU_DT", to_date(col("CLM_THRU_DT"), "yyyyMMdd"))
    .withColumn("NCH_BENE_DSCHRG_DT", to_date(col("NCH_BENE_DSCHRG_DT"), "yyyyMMdd"))
    .fillna("unknown")
)

In [0]:
inpatient_clean.write.mode("overwrite").parquet("/mnt/silver/inpatient/inpatient_clean")

In [0]:
from pyspark.sql.functions import col, year, when, to_date, lit

# Inpatient Silver Table
inpatient_silver = inpatient_clean.filter(
    col("clm_admsn_dt").isNotNull()
).select(
    "desynpuf_id", "clm_id", "clm_admsn_dt", "clm_drg_cd",
    "nch_bene_dschrg_dt", "admtng_icd9_dgns_cd", "icd9_dgns_cd_1"
)

In [0]:
inpatient_silver.write.mode("overwrite").parquet("/mnt/silver/inpatient/inpatient_silver")